<a href="https://colab.research.google.com/github/cytric-74/gta6-sales-prediction-/blob/main/gta6_sales_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.8 MB/s eta 0:00:00


In [8]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from googleapiclient.discovery import build
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import numpy as np

In [5]:
!mkdir data


# ***Twitter data collection***

In [ ]:
TOKKEN_TWIITER = "we need a token here"

In [ ]:
def fetch_twitter_data(query, max_results=50):
    headers = {"Authorization": f"Bearer {TOKKEN_TWIITER}"}
    url = f"https://api.twitter.com/2/tweets/search/recent?query={query}&tweet.fields=created_at,public_metrics&max_results={max_results}"
    response = requests.get(url, headers=headers)
    tweets = response.json()('data', [])

    df = pd.DataFrame([{
        'text': tweet['text'],
        'created_at': tweet['created_at'],
        'likes': tweet['public_metrics']['like_count']
    } for tweet in tweets])

    df.to_csv('data/twitter_data.csv', index=False)
    return df

# ***Youtube data collection***

In [6]:
from googleapiclient.discovery import build
import pandas as pd

In [ ]:
API_KEY = 'need to get a youtube api key asap'
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [ ]:
def fetch_youtube_comments(query):
    search_response = youtube.search().list(
        q=query, part='snippet', type='video', maxResults=5
    ).execute()

    video_ids = [item['id']['videoId'] for item in search_response['items']]
    all_comments = []

    for vid in video_ids:
        comment_response = youtube.commentThreads().list(
            part='snippet', videoId=vid, textFormat='plainText', maxResults=50
        ).execute()

        for item in comment_response.get("items", []):
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            all_comments.append(comment)

    df = pd.DataFrame({'comment': all_comments})
    df.to_csv('data/youtube_data.csv', index=False)
    return df

# ***Web Scrapping***

---



In [ ]:
def scrape_news(query):
    formatted_query = query.replace(" ", "%20")
    search_url = f"https://news.google.com/search?q={formatted_query}"
    response = requests.get(search_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = []
    for a in soup.select('article h3'):
        if a.a:
            title = a.text.strip()
            link = "https://news.google.com" + a.a['href'][1:]
            articles.append({'title': title, 'link': link})

    df = pd.DataFrame(articles)
    df.to_csv('data/news_articles.csv', index=False)
    return df

# ***Sentiment Analysis***

In [ ]:
def analyze_sentiment(df, column):
    df['sentiment'] = df[column].astype(str).apply(lambda x: analyzer.polarity_scores(x)['compound'])
    return df

# ***Model Prediction***

In [ ]:
def combine_and_predict():
    twitter_df = analyze_sentiment(pd.read_csv("data/twitter_data.csv"), "text")
    youtube_df = analyze_sentiment(pd.read_csv("data/youtube_data.csv"), "comment")
    news_df = analyze_sentiment(pd.read_csv("data/news_articles.csv"), "title")

    avg_sentiments = pd.DataFrame({
        'twitter_sentiment': [twitter_df['sentiment'].mean()],
        'youtube_sentiment': [youtube_df['sentiment'].mean()],
        'news_sentiment': [news_df['sentiment'].mean()]
    })

    # Dummy training data with similar structure (for demonstration)
    X_train = pd.DataFrame({
        'twitter_sentiment': np.random.uniform(-1, 1, 100),
        'youtube_sentiment': np.random.uniform(-1, 1, 100),
        'news_sentiment': np.random.uniform(-1, 1, 100),
    })
    y_train = X_train.mean(axis=1) * 50 + 85  # Just a mocked-up sales range

    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'MLP Regressor': MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
    }

    predictions = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        pred = model.predict(avg_sentiments)[0]
        predictions[name] = round(pred, 2)
        print(f"{name} Predicted GTA 6 Sales (in millions):", round(pred, 2))

    result_df = avg_sentiments.copy()
    for name, pred in predictions.items():
        result_df[name.replace(" ", "_").lower() + '_prediction'] = pred

    result_df.to_csv("data/sales_prediction.csv", index=False)

# ***Running***

In [ ]:
if __name__ == "__main__":
    print("Collecting Twitter data...")
    fetch_twitter_data("GTA 6")

    print("Collecting YouTube comments...")
    fetch_youtube_comments("GTA 6 trailer")

    print("Scraping news articles...")
    scrape_news("GTA 6")

    print("Analyzing sentiment and predicting sales with multiple models...")
    combine_and_predict()
    print("All done! Check the 'data/' folder for CSVs.")